In [ ]:
import numpy as np
from numpy import linalg as la

In [ ]:
from sympy import Matrix

In [ ]:
from sympy.polys.matrices import DomainMatrix

In [ ]:
'''
b : basis vector
b_star : orthogonalized vector
delta : Lovasz constant, value we'll be working with -> 0.75
k : keeps track of the index being worked on

'''

"\nb : basis vector\nb_star : orthogonalized vector\ndelta : Lovasz constant, value we'll be working with -> 0.75\nk : keeps track of the index being worked on\n\n"

In [ ]:
def create_lattice_matrix(h, N, q):
    H = np.zeros((N, N), dtype=int)
    for i in range(N):
        H[i] = np.roll(h, i)
    I = np.eye(N, dtype=int)
    M_upper = np.hstack((I, H))
    M_lower = np.hstack((np.zeros((N, N), dtype=int), q * I))
    return np.vstack((M_upper, M_lower))

In [ ]:
def map_to_range(a):
  p=3
  print(a)
  return np.array([(x if x <= p // 2 else x - p) for x in a])

In [ ]:


# Initialize the basis as a user input (example).
#b = np.array([[1, 1, 1], [-1, 0, 2], [3, 5, 6]]).astype(float)
b = np.array(create_lattice_matrix([8, 37, 24, 32, 6, 38, 19],7,41)).astype(float)
print("b= ",b)
b_star = b.copy()   # Initialize the Gram-Schmidt basis.

k = 1  # Initialize the working index.
delta = 0.75  # Lovasz constant

def mu(u, v):
    '''Computes <u,v>/<u,u>, which is the scale used in projection.'''
    return np.dot(u, v) / np.dot(u, u)

def proj(u, v):
    '''Computes the projection of vector v onto vector u. Assumes u is not zero.'''
    return mu(u, v) * u

def gram_schmidt():
    '''Computes Gram Schmidt orthogonalization of a basis.'''
    b_star[0] = b[0]
    for i in range(1, b.shape[0]):  # Loop through dimension of basis.
        b_star[i] = b[i]
        for j in range(0, i):
            b_star[i] -= proj(b_star[j], b[i])
    return b_star

def reduction():
    '''Performs length reduction on a basis.'''
    global k
    total_reduction = 0  # Track the total amount by which the working vector is reduced.
    for j in range(k-1, -1, -1):   # j loop. Loop down from k-1 to 0.
        m = round(mu(b_star[j], b[k]))
        total_reduction += m * b[j][0]
        b[k] -= m * b[j]  # Reduce the working vector by multiples of preceding vectors.
    if total_reduction > 0:
        gram_schmidt()  # Recompute Gram-Schmidt if the working vector has been reduced.

def lovasz():
    global k
    '''Checks the Lovasz condition for a basis. Either swaps adjacent basis vectors and recomputes Gram-Schmidt or increments the working index.'''
    c = delta - mu(b_star[k-1], b[k])**2
    if la.norm(b_star[k])**2 >= (c * la.norm(b_star[k-1])**2):  # Check the Lovasz condition.
        k += 1  # Increment k if the condition is met.
    else:
        b[[k, k-1]] = b[[k-1, k]]  # Swap the working vector and the immediately preceding basis vector.
        gram_schmidt()  # Recompute Gram-Schmidt if swapped.
        k = max([k-1, 1])

def lll():
    global k
    gram_schmidt()
    steps = 0
    while k <= b.shape[0] - 1:
        reduction()
        steps += 1


        lovasz()
        steps += 1


    print('LLL Reduced Basis:\n', b)
    print('No of. steps in calculation: ', steps)


lll()




b=  [[ 1.  0.  0.  0.  0.  0.  0.  8. 37. 24. 32.  6. 38. 19.]
 [ 0.  1.  0.  0.  0.  0.  0. 19.  8. 37. 24. 32.  6. 38.]
 [ 0.  0.  1.  0.  0.  0.  0. 38. 19.  8. 37. 24. 32.  6.]
 [ 0.  0.  0.  1.  0.  0.  0.  6. 38. 19.  8. 37. 24. 32.]
 [ 0.  0.  0.  0.  1.  0.  0. 32.  6. 38. 19.  8. 37. 24.]
 [ 0.  0.  0.  0.  0.  1.  0. 24. 32.  6. 38. 19.  8. 37.]
 [ 0.  0.  0.  0.  0.  0.  1. 37. 24. 32.  6. 38. 19.  8.]
 [ 0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.]]
LLL Reduced Basis:
 [[ 1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0. -1.  1.  0. -1. -1. -3.  0. -3.  0.  3.  3.  0.]
 [ 1.  0. -1. -1.  1.  0. -1.  

In [ ]:
def construct_ntru_lattice(h, N, q):
    H = np.zeros((N, N), dtype=int)
    for i in range(N):
        H[i] = np.roll(h, i)

    lattice_matrix = np.block([
        [np.identity(N, dtype=int), H],
        [np.zeros((N, N), dtype=int), q * np.identity(N, dtype=int)]
    ])

    return Matrix(lattice_matrix)
ans=construct_ntru_lattice([8, 37, 24, 32, 6, 38, 19],7,41)

In [ ]:
print(ans)

Matrix([[1, 0, 0, 0, 0, 0, 0, 8, 37, 24, 32, 6, 38, 19], [0, 1, 0, 0, 0, 0, 0, 19, 8, 37, 24, 32, 6, 38], [0, 0, 1, 0, 0, 0, 0, 38, 19, 8, 37, 24, 32, 6], [0, 0, 0, 1, 0, 0, 0, 6, 38, 19, 8, 37, 24, 32], [0, 0, 0, 0, 1, 0, 0, 32, 6, 38, 19, 8, 37, 24], [0, 0, 0, 0, 0, 1, 0, 24, 32, 6, 38, 19, 8, 37], [0, 0, 0, 0, 0, 0, 1, 37, 24, 32, 6, 38, 19, 8], [0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 41]])


In [ ]:
dM = DomainMatrix.from_Matrix(ans)

In [ ]:
dM.lll().to_Matrix()

Matrix([
[ 1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0],
[ 1,  0, -1,  1,  0, -1, -1, -3,  0, -3,  0,  3,  3,  0],
[ 1,  0, -1, -1,  1,  0, -1,  0,  3,  3,  0, -3,  0, -3],
[-1,  1,  0, -1,  1,  0, -1,  0, -3,  0, -3,  0,  3,  3],
[-1,  1,  0, -1, -1,  1,  0, -3,  0,  3,  3,  0, -3,  0],
[-1,  0,  0,  0, -2,  1, -1, -3,  0, -3,  3,  0,  0,  3],
[-2,  1, -1, -1,  0,  0,  0,  0,  0,  3, -3,  0, -3,  3],
[-1,  0,  5, -2, -1,  2, -5,  1,  2, -3,  0,  3, -1, -2],
[ 5, -2, -1,  2, -5, -1,  0, -3,  0,  3, -1, -2,  1,  2],
[ 0, -5,  2,  1, -2,  5,  1, -2,  3,  0, -3,  1,  2, -1],
[-4, -3,  0,  3,  3,  5, -5,  1,  0, -2,  0,  2, -1,  0],
[-3,  4, -2,  5,  1, -5,  0,  0,  1, -1,  0, -4,  4,  0],
[-2,  5,  1,  0, -5,  2,  1,  1,  2, -1, -2,  3,  0, -3],
[ 2,  0,  0, -3,  2, -4,  0,  7,  8,  2,  8,  5,  5,  6]])